In [27]:
### Use with quandl environment

import json
import requests
import nest_asyncio
from datetime import datetime
import ipywidgets as widgets
import pandas as pd
import gmaps
import numpy as np
import warnings


warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 0)
#pd.set_option("display.max_rows", 10)

nest_asyncio.apply()

myAPI='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
gmaps.configure(api_key=myAPI)


def GetLocation():    
    className="GeoTracker"
    url="xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
    url=url+className

    header = {"X-Parse-Application-Id":"xxxxxxxxxxxxxxxxxxxxxxxxxx","X-Parse-REST-API-Key":"undefined"}
    param={"limit":"1000"}
    response = requests.get(url,data=param, headers=header)
    result=response.json()

    df = pd.DataFrame.from_dict(result)

    dfCom=pd.DataFrame(list(df.iloc[0]))
    for n in range(1,len(df)):
        dfCom=dfCom.append(pd.DataFrame(list(df.iloc[n])),ignore_index = True)
    
    return dfCom


def MakeMap(dfAcc_1):

    locations = dfAcc_1[['lat', 'lon']].copy()

    locStart=tuple(list(locations.iloc[0]))
    locEnd=tuple(list(locations.iloc[-1]))

    latMean=locations['lat'].mean()
    lngMean=locations['lon'].mean()


    maxSpeed=dfAcc_1['Speed'].max()
    dfAcc_1['Speed']=dfAcc_1['Speed'].astype(float)

    def AssignColor(speedRatio):    
        if(speedRatio<0.2):
            color='black'
        elif(speedRatio>=0.2 and speedRatio<0.5):
            color='green'
        elif(speedRatio>=0.5 and speedRatio<0.8):
            color='yellow'
        else:
            color='red'
        return color

    def CalSpeedRatio(inVal, maxSpeed):    
        return inVal/maxSpeed

    dfAcc_1['speedRatio']=dfAcc_1.apply(lambda x: CalSpeedRatio(x['Speed'],maxSpeed),axis=1)
    dfAcc_1['color']=dfAcc_1.apply(lambda x: AssignColor(x['speedRatio']),axis=1)
    colors=dfAcc_1['color'].values.tolist()


    figure_layout = {
        'width': '600px',
        'height': '400px',
        'border': '1px solid black',
        'padding': '1px'
    }

    #bangkok_coordinates = (13.7464, 100.646)
    bangkok_coordinates = (latMean, lngMean)
    fig = gmaps.figure(center=bangkok_coordinates,zoom_level=15, layout=figure_layout)
    
    latList=dfAcc_1['lat'].values.tolist()
    lonList=dfAcc_1['lon'].values.tolist()
    llData=list(zip(latList,lonList))
    labelList=[]
    for n in range(0,len(dfAcc_1)):
        labelList.append({"name":dfAcc_1['Name'][n], "datetime":dfAcc_1['Date'][n],
        "location":(dfAcc_1['lat'][n],dfAcc_1['lon'][n]), "speed":dfAcc_1['Speed'][n]})
        
    info_box_template = """
    <dl>
    <dt><b><p style="color:red;">username<p></b></dt><dd><p style="color:red;">{name}<p></dd>
    <dt><b><p style="color:red;">datetime<p></b></dt><dd><p style="color:red;">{datetime}<p></dd>
    <dt><b><p style="color:red;">tweet<p></b></dt><dd><p style="color:red;">{speed}<p></dd>
    </dl>
    """

    plant_info = [info_box_template.format(**plant) for plant in labelList]

    info_box_template_2 = """
    name : {name}
    datetime : {datetime}
    speed    :{speed}
    """
    plant_info_2 = [info_box_template_2.format(**plant) for plant in labelList]

    #markerLayer=gmaps.marker_layer(llData, info_box_content=plant_info, hover_text=plant_info_2)
    #fig.add_layer(markerLayer)

    incLayer = gmaps.symbol_layer(
        locations, info_box_content=plant_info, hover_text=plant_info_2, fill_color=colors, stroke_color=colors, scale=2
    )
    fig.add_layer(incLayer)
    fig.add_layer(gmaps.traffic_layer())

    #directions = gmaps.directions_layer(locStart, locEnd)
    #fig.add_layer(directions)

    locList=[]
    for n in range(1,len(locations)):
        locStart=tuple(list(locations.iloc[n-1]))
        locEnd=tuple(list(locations.iloc[n]))
        dummy=gmaps.Line(locStart,locEnd, stroke_color="blue",stroke_weight=5.0)
        locList.append(dummy)

    drawing = gmaps.drawing_layer(features=locList)
    fig.add_layer(drawing)    
    return fig



def Diplay(dfAcc2):
    displayFlag=0
    try:
        fig=MakeMap(dfAcc2)
        displayFlag=1
        display(fig)
    except:
        print(' No Data ')
        displayFlag=0
    #fig=MakeMap(dfAcc2)
    #displayFlag=1
    #display(fig)

dfAcc1=GetLocation()
dfAcc1['Date'] = pd.to_datetime(dfAcc1['Date'])
minDate=dfAcc1['Date'].min()
maxDate=dfAcc1['Date'].max()

start_date = minDate
end_date = maxDate

dates = pd.date_range(start_date, end_date, freq='min')

options = [(date.strftime('%H:%M %d/%m'), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Time',
    orientation='horizontal',
    layout={'width': '500px'}
)

def print_date_range(date_range):
    #print(date_range)    
    startTime=date_range[0]
    endTime=date_range[1]
    
    #print(date_range[0], " ---  ",date_range[1])
    dfAcc1=GetLocation()
    dfAcc1['Date'] = pd.to_datetime(dfAcc1['Date'])
    #print(dfAcc1)
    dfAcc2=dfAcc1[(dfAcc1['Date']>=startTime) & (dfAcc1['Date']<=endTime)].reset_index().copy() 
    #print(dfAcc2)
    Diplay(dfAcc2)
    #print('1')
    sumDistance=dfAcc2['Distance'].sum(skipna = True) 
    print(" Traveling Distance :  {0:.2f} km  ".format(sumDistance/1000))

widgets.interact(
    print_date_range,
    date_range=selection_range_slider
)


interactive(children=(SelectionRangeSlider(description='Time', index=(0, 37), layout=Layout(width='500px'), op…

<function __main__.print_date_range(date_range)>